In [1]:
from llama_cpp import Llama

MODEL_PATH = "../models/Phi-3-mini-4k-instruct-Q4_K_M/Phi-3-mini-4k-instruct-Q4_K_M.gguf"

llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=4096,        # full context của Phi-3 Mini
    n_threads=12,
    verbose=False
)

In [2]:
rag_chunks = [
    "AI là lĩnh vực khoa học máy tính nghiên cứu cách tạo ra hệ thống thông minh có khả năng học hỏi, suy luận và tự giải quyết vấn đề tương tự như trí tuệ con người.",
    "Các ứng dụng phổ biến của AI bao gồm nhận dạng hình ảnh, xử lý ngôn ngữ tự nhiên, hệ thống gợi ý cá nhân hóa và điều khiển xe tự hành trong giao thông.",
    "Hạn chế của AI hiện nay gồm phụ thuộc dữ liệu lớn để huấn luyện, chi phí tính toán cao và khả năng giải thích các quyết định phức tạp còn hạn chế (vấn đề hộp đen)."
]

In [3]:
import sys

# ==============================
# INPUT: RAG CHUNKS
# ==============================
rag_chunks = [
    "AI là lĩnh vực khoa học máy tính nghiên cứu cách tạo ra hệ thống thông minh có khả năng học hỏi, suy luận và tự giải quyết vấn đề tương tự như trí tuệ con người.",
    "Các ứng dụng phổ biến của AI bao gồm nhận dạng hình ảnh, xử lý ngôn ngữ tự nhiên, hệ thống gợi ý cá nhân hóa và điều khiển xe tự hành trong giao thông.",
    "Hạn chế của AI hiện nay gồm phụ thuộc dữ liệu lớn để huấn luyện, chi phí tính toán cao và khả năng giải thích các quyết định phức tạp còn hạn chế (vấn đề hộp đen)."
]

# ==============================
# BUILD RAG PROMPT
# ==============================
prompt = f"""
<|user|>
Dựa trên các tài liệu tham khảo dưới đây, hãy:
- Giải thích lại bằng tiếng Việt chuẩn, mạch lạc, học thuật
- Có thể diễn đạt lại cho rõ nghĩa nhưng không thêm thông tin ngoài tài liệu

### TÀI LIỆU THAM KHẢO
[1] {rag_chunks[0]}
[2] {rag_chunks[1]}
[3] {rag_chunks[2]}

### CÂU HỎI
AI là gì? Nêu ví dụ, ứng dụng thực tế và các hạn chế.
<|assistant|>
"""

# ==============================
# HELPER FUNCTIONS
# ==============================
def ends_with_sentence(text):
    return text.strip().endswith((".", "!", "?"))

def judge_answer(llm, answer):
    judge_prompt = f"""
<|user|>
Đánh giá câu trả lời sau đã ĐẦY ĐỦ hay CHƯA.

Tiêu chí:
1. Có định nghĩa AI
2. Có ứng dụng thực tế
3. Có hạn chế của AI

Chỉ trả lời đúng một từ:
YES hoặc NO.

### CÂU TRẢ LỜI
{answer}
<|assistant|>
"""
    out = llm(
        judge_prompt,
        max_tokens=5,
        temperature=0.0
    )
    verdict = out["choices"][0]["text"].strip().upper()
    print(f"\n[JUDGE] => {verdict}\n")
    return verdict == "YES"

# ==============================
# STEP 1: GENERATE + STREAMING
# ==============================
print("===== GENERATING ANSWER =====\n")

response = ""

for chunk in llm(
    prompt,
    max_tokens=1024,
    temperature=0.3,
    top_p=0.9,
    stream=True
):
    token = chunk["choices"][0]["text"]
    if token:
        response += token
        sys.stdout.write(token)
        sys.stdout.flush()

# ==============================
# STEP 2: AUTO-CONTINUE TO FINISH SENTENCE
# ==============================
if not ends_with_sentence(response):
    continue_prompt = f"""
<|user|>
Hãy hoàn thành câu cuối cùng sau cho trọn vẹn, không thêm ý mới:

{response}
<|assistant|>
"""
    for chunk in llm(
        continue_prompt,
        max_tokens=64,
        temperature=0.3,
        top_p=0.9,
        stream=True
    ):
        token = chunk["choices"][0]["text"]
        if token:
            response += token
            sys.stdout.write(token)
            sys.stdout.flush()

# ==============================
# STEP 3: LLM-AS-A-JUDGE
# ==============================
if not judge_answer(llm, response):
    print("[INFO] Câu trả lời chưa đủ, đang bổ sung phần còn thiếu...\n")

    supplement_prompt = f"""
<|user|>
Câu trả lời sau CHƯA đầy đủ.
Hãy bổ sung PHẦN CÒN THIẾU để đáp ứng đủ:
- định nghĩa
- ứng dụng
- hạn chế

KHÔNG lặp lại nội dung đã có.

### CÂU TRẢ LỜI HIỆN TẠI
{response}
<|assistant|>
"""

    for chunk in llm(
        supplement_prompt,
        max_tokens=256,
        temperature=0.3,
        top_p=0.9,
        stream=True
    ):
        token = chunk["choices"][0]["text"]
        if token:
            response += token
            sys.stdout.write(token)
            sys.stdout.flush()

# ==============================
# FINAL OUTPUT
# ==============================
print("\n\n===== FINAL ANSWER (COMPLETE) =====\n")
print(response)


===== GENERATING ANSWER =====

 AI là một lĩnh vực khoa học mạch lạc, thể hiện trong các hệ thống thông minh có khả năng học hỏi,

KeyboardInterrupt: 

In [ ]:
def ends_with_sentence(text):
    return text.strip().endswith((".", "!", "?"))

response = ""

# lượt sinh chính
for chunk in llm(prompt, max_tokens=1024, stream=True):
    token = chunk["choices"][0]["text"]
    if token:
        response += token
        print(token, end="", flush=True)

# nếu chưa tròn câu → gọi tiếp
if not ends_with_sentence(response):
    fix_prompt = f"""
<|user|>
Hãy hoàn thành câu sau cho trọn vẹn, không thêm ý mới:
{response}
<|assistant|>
"""
    for chunk in llm(fix_prompt, max_tokens=64, stream=True):
        token = chunk["choices"][0]["text"]
        if token:
            print(token, end="", flush=True)